<a href="https://colab.research.google.com/github/ahmadfadhilnugraha/Zuber/blob/main/Sprint_6_Zuber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zuber - Preferencias de los pasajeros y la influencia del clima en los viajes

## 1. Preprocesamiento de datos

### Análisis sintáctico de datos

Escribe un código para analizar los datos sobre el clima en Chicago en noviembre de 2017 desde el sitio web:

[https://practicum-content.s3.us-west-1.amazonaws.com/data-analyst-eng/moved_chicago_weather_2017.html)

El nombre del DataFrame debe ser weather_records y tienes que especificarlo cuando buscas: attrs={"id": "weather_records"} . Imprime el DataFrame completo.

In [26]:
import requests  # Importa la librería para enviar solicitudes al servidor
from bs4 import BeautifulSoup # importa la librería beautiful soup
import pandas as pd

req = requests.get('https://practicum-content.s3.us-west-1.amazonaws.com/data-analyst-eng/moved_chicago_weather_2017.html') # actúa como un navegador. 

#print(f'Estado de la página: {req.status_code}') # conocer el estado de la página web, 200 es todo OK

soup = BeautifulSoup(req.text, 'lxml')

table = soup.find('table', attrs={"id": "weather_records"})

heading_table = []
for i in table.find_all('th'):
    heading_table.append(i.text)

content = []
for i in table.find_all('tr'):
    if not i.find_all('th'):
        content.append([element.text for element in i.find_all('td')])

weather_records = pd.DataFrame(content, columns= heading_table)

print(weather_records)

           Date and time Temperature       Description
0    2017-11-01 00:00:00     276.150     broken clouds
1    2017-11-01 01:00:00     275.700  scattered clouds
2    2017-11-01 02:00:00     275.610   overcast clouds
3    2017-11-01 03:00:00     275.350     broken clouds
4    2017-11-01 04:00:00     275.240     broken clouds
..                   ...         ...               ...
692  2017-11-29 20:00:00     281.340        few clouds
693  2017-11-29 21:00:00     281.690      sky is clear
694  2017-11-29 22:00:00     281.070        few clouds
695  2017-11-29 23:00:00     280.060      sky is clear
696  2017-11-30 00:00:00     278.460      sky is clear

[697 rows x 3 columns]


### Trabajo con base de datos

**1. Imprime el campo company_name. Encuentra la cantidad de viajes en taxi para cada compañía de taxis para el 15 y 16 de noviembre de 2017, asigna al campo resultante el nombre trips_amount e imprímelo también. Ordena los resultados por el campo trips_amount en orden descendente.**

In [27]:
'''
SELECT
    cabs.company_name,
    COUNT(trips.trip_id) AS trips_amount
FROM
    cabs
    INNER JOIN
    trips
    ON
    trips.cab_id = cabs.cab_id
WHERE
    CAST(trips.start_ts AS date) BETWEEN '2017-11-15' AND '2017-11-16'
GROUP BY
    company_name
ORDER BY
    trips_amount DESC;
'''

"\nSELECT\n    cabs.company_name,\n    COUNT(trips.trip_id) AS trips_amount\nFROM\n    cabs\n    INNER JOIN\n    trips\n    ON\n    trips.cab_id = cabs.cab_id\nWHERE\n    CAST(trips.start_ts AS date) BETWEEN '2017-11-15' AND '2017-11-16'\nGROUP BY\n    company_name\nORDER BY\n    trips_amount DESC;\n"

**2. Encuentra la cantidad de viajes para cada empresa de taxis cuyo nombre contenga las palabras "Yellow" o "Blue" del 1 al 7 de noviembre de 2017. Nombra la variable resultante trips_amount. Agrupa los resultados por el campo company_name.**

In [28]:
'''
SELECT
    cabs.company_name as company_name,
    COUNT(trips.trip_id) AS trips_amount
FROM
    cabs
INNER JOIN
    trips
ON
    trips.cab_id = cabs.cab_id
WHERE
    CAST(trips.start_ts AS date) BETWEEN '2017-11-01' AND '2017-11-07'
    AND cabs.company_name LIKE '%%Yellow%%'
GROUP BY company_name
UNION ALL
SELECT
    cabs.company_name as company_name,
    COUNT(trips.trip_id) AS trips_amount
FROM
    cabs
INNER JOIN
    trips
ON
    trips.cab_id = cabs.cab_id
WHERE
    CAST(trips.start_ts AS date) BETWEEN '2017-11-01' AND '2017-11-07'
    AND cabs.company_name LIKE '%%Blue%%'
GROUP BY company_name;\
'''

"\nSELECT\n    cabs.company_name as company_name,\n    COUNT(trips.trip_id) AS trips_amount\nFROM\n    cabs\nINNER JOIN\n    trips\nON\n    trips.cab_id = cabs.cab_id\nWHERE\n    CAST(trips.start_ts AS date) BETWEEN '2017-11-01' AND '2017-11-07'\n    AND cabs.company_name LIKE '%%Yellow%%'\nGROUP BY company_name\nUNION ALL\nSELECT\n    cabs.company_name as company_name,\n    COUNT(trips.trip_id) AS trips_amount\nFROM\n    cabs\nINNER JOIN\n    trips\nON\n    trips.cab_id = cabs.cab_id\nWHERE\n    CAST(trips.start_ts AS date) BETWEEN '2017-11-01' AND '2017-11-07'\n    AND cabs.company_name LIKE '%%Blue%%'\nGROUP BY company_name;"

**3. Del 1 al 7 de noviembre de 2017, las empresas de taxis más populares fueron Flash Cab y Taxi Affiliation Services. Encuentra el número de viajes de estas dos empresas y asigna a la variable resultante el nombre trips_amount. Junta los viajes de todas las demás empresas en el grupo "Other". Agrupa los datos por nombres de empresas de taxis. Asigna el nombre company al campo con nombres de empresas de taxis. Ordena el resultado en orden descendente por trips_amount.**


In [29]:
'''
SELECT
    CASE
        WHEN company_name = 'Flash Cab' THEN 'Flash Cab'
        WHEN company_name = 'Taxi Affiliation Services' THEN 'Taxi Affiliation Services'
        ELSE 'Other'
    END AS company,
    COUNT(trips.trip_id) as trips_amount
FROM
    cabs
INNER JOIN
    trips
ON
    trips.cab_id = cabs.cab_id
WHERE
    CAST(trips.start_ts AS date) BETWEEN '2017-11-01' AND '2017-11-07'
GROUP BY
    company
ORDER BY
    trips_amount DESC;
'''

"\nSELECT\n    CASE\n        WHEN company_name = 'Flash Cab' THEN 'Flash Cab'\n        WHEN company_name = 'Taxi Affiliation Services' THEN 'Taxi Affiliation Services'\n        ELSE 'Other'\n    END AS company,\n    COUNT(trips.trip_id) as trips_amount\nFROM\n    cabs\nINNER JOIN\n    trips\nON\n    trips.cab_id = cabs.cab_id\nWHERE\n    CAST(trips.start_ts AS date) BETWEEN '2017-11-01' AND '2017-11-07'\nGROUP BY\n    company\nORDER BY\n    trips_amount DESC;\n"

**4. Recupera los identificadores de los barrios de O'Hare y Loop de la tabla neighborhoods.**

In [30]:
'''
SELECT
    neighborhood_id,
    name
FROM
    neighborhoods
WHERE
    name LIKE '%Hare' OR name LIKE 'Loop'
'''

"\nSELECT\n    neighborhood_id,\n    name\nFROM\n    neighborhoods\nWHERE\n    name LIKE '%Hare' OR name LIKE 'Loop'\n"

**5. Para cada hora recupera los registros de condiciones meteorológicas de la tabla weather_records. Usando el operador CASE, divide todas las horas en dos grupos: Bad si el campo description contiene las palabras rain o storm, y Good para los demás. Nombra el campo resultante weather_conditions. La tabla final debe incluir dos campos: fecha y hora (ts) y weather_conditions.**

In [31]:
'''
SELECT
    ts,
    CASE
        WHEN description LIKE '%rain%' OR description LIKE '%storm%' THEN 'Bad'
        ELSE 'Good'
    END AS weather_conditions
FROM
    weather_records;
'''

"\nSELECT\n    ts,\n    CASE\n        WHEN description LIKE '%rain%' OR description LIKE '%storm%' THEN 'Bad'\n        ELSE 'Good'\n    END AS weather_conditions\nFROM\n    weather_records;\n"

**6. Recupera de la tabla de trips todos los viajes que comenzaron en el Loop (pickup_location_id: 50) el sábado y terminaron en O'Hare (dropoff_location_id: 63). Obtén las condiciones climáticas para cada viaje. Utiliza el método que aplicaste en la tarea anterior. Recupera también la duración de cada viaje. Ignora los viajes para los que no hay datos disponibles sobre las condiciones climáticas**

In [32]:
'''
SELECT
    start_ts,
    T.weather_conditions,
    duration_seconds
FROM
    trips
INNER JOIN (
    SELECT
        ts,
        CASE
            WHEN description LIKE '%rain%' OR description LIKE '%storm%' THEN 'Bad'
            ELSE 'Good'
        END AS weather_conditions
    FROM
        weather_records
) T on T.ts = trips.start_ts
WHERE
    pickup_location_id = 50 AND dropoff_location_id = 63 AND EXTRACT (DOW from trips.start_ts) = 6
ORDER BY trip_id
'''

"\nSELECT\n    start_ts,\n    T.weather_conditions,\n    duration_seconds\nFROM\n    trips\nINNER JOIN (\n    SELECT\n        ts,\n        CASE\n            WHEN description LIKE '%rain%' OR description LIKE '%storm%' THEN 'Bad'\n            ELSE 'Good'\n        END AS weather_conditions\n    FROM\n        weather_records\n) T on T.ts = trips.start_ts\nWHERE\n    pickup_location_id = 50 AND dropoff_location_id = 63 AND EXTRACT (DOW from trips.start_ts) = 6\nORDER BY trip_id\n"

Después de este proceso, tendremos tres nuevos datasets.
1. project_sql_result_01.csv. Este archivo contiene los siguientes datos:
    - company_name: nombre de la empresa de taxis
    - trips_amount: el número de viajes de cada compañía de taxis el 15 y 16 de noviembre de 2017. 
2. project_sql_result_04.csv. Este archivo contiene los siguientes datos:
    - dropoff_location_name: barrios de Chicago donde finalizaron los viajes
    - average_trips: el promedio de viajes que terminaron en cada barrio en noviembre de 2017.
3. project_sql_result_07.csv. Este archivo contiene los datos de viajes del barrio Loop al aeropuerto internacional O'Hare:
    - start_ts: fecha y hora de la recogida
    - weather_conditions: condiciones climáticas en el momento en el que comenzó el viaje
    - duration_seconds: duración del viaje en segundos